## Average Distance

**Requirements**
In order to run this notebook you will need:
* `yelp`, `pandas`, `folium`, `geojson`, `numpy`, `requests`, `geopy`, and `bokeh` libraries (all available via `pip`).
* A [Yelp! API key](https://www.yelp.com/developers/manage_api_keys), saved to a credentials file in the format recommended [here](https://github.com/Yelp/yelp-python).

**Notes**
* [Reference](http://nbviewer.jupyter.org/github/ResidentMario/notebooks/blob/master/gis-viz-in-python.ipynb).
* Google Maps has [super duper shapefiles](https://www.google.com/maps/place/Manhattan,+New+York,+NY/@40.7858205,-74.0272543,12z/data=!4m2!3m1!1s0x89c2588f046ee661:0xa0b3281fcecc08c) but their shapefile database is not public.
* The best precompiled city shapefile repository is [this one](https://github.com/johan/world.geo.json). However, the city shapefiles *suck*. Some of the country ones do to, amazingly! What happened!
* We can't use census tracts because census areas and city designations are two different things. But Bostock has an [atlas](https://github.com/mbostock/us-atlas) built using public-source data. [Here](http://bl.ocks.org/mbostock/5349951) is a demo. Just not what I need in this specific case.
* London, New York, and Tokyo are three cities which I have a good GeoJSON repr for, thanks to [this](https://github.com/utisz/compound-cities).
* Note that GeoJSON is not good for complex representations, [TopoJSON](https://github.com/mbostock/topojson) uses arcs and is better for this case.
* [This](http://stackoverflow.com/questions/22440086/topojson-data-for-all-us-cities) points to a way to generate midpoints of locations easily. Not quite what I need but it would be useful for general map-building. Although maybe better to just frame the datapoints?
* **Working assumption**: I've figured out and generates the neceessay GeoJSON shape files for my cities of interest (atm only have New York City though! Working with Manhattan to start with).
* How to generate a list of chains in the city?
* Maybe use a no-parameters Google API radar search, spool up several and crunch the incidence rate?
* Find a centralized list of them?

In [2]:
from yelp.client import Client
from yelp.oauth1_authenticator import Oauth1Authenticator
from yelp.errors import BusinessUnavailable
import os
import json
from pandas import DataFrame
import folium
import geojson
import random
import requests
import numpy as np
from geopy.distance import vincenty
import bokeh.plotting as plt

In [11]:
def import_credentials(filename='yelp_credentials.json'):
    """
    Finds the credentials file describing the token that's needed to access Yelp services.

    :param filename -- The filename at which Yelp service credentials are stored. Defaults to
    `yelp_credentials.json`.
    """
    if filename in [f for f in os.listdir('.') if os.path.isfile(f)]:
        data = json.load(open(filename))
        return data
    else:
        raise IOError('This API requires Yelp credentials to work. Did you forget to define them?')

def fetch_businesses(name, area='New York', manual_override=0):
    area = area.lower().replace(' ', '-')
    name = name.lower().replace(' ', '-')
    """
    Fetches all yelp.obj.business_response.BusinessResponse objects for incidences of the given chain in Manhattan.
    Constructs Yelp business ids for incidences of the chain in the area, then queries Yelp to check if they
    exist.
    IDs are constructed name-location-number, so we just have to check numbers in ascending order until it breaks.
    e.g. http://www.yelp.com/biz/gregorys-coffee-new-york-18 is good.
         http://www.yelp.com/biz/gregorys-coffee-new-york-200 is not.
    Then we do reverse GIS searches using the business ID through the Yelp API and extract coordinates from the results.
    Some technical notes:
    1.  The first incidence of any store in the area is reported without any numeral.
        e.g. "dunkin-donuts-new-york", not "dunkin-donuts-new-york-1".
        Numbers pick up from there: the next shitty hole in the wall will be "dunkin-donuts-new-york-2".
    2.  Yelp IDs are unique and are not reassigned when a location is closed.
        Thus we need to check for and exclude closed locations when munging the data.
    3.  Places with a single instance in Manhattan sometimes have a "name-place-2" that redirects to their only location.
        At least this seems to be the case with Bibble & Sip...
        This is checked and corrected for further down the line, by the fetch_businesses() method.
    4.  Sometimes IDs are given to locations that don't actually really exist.
        e.g. the best-buy-3 id points to a non-existant storefront.
        But best-buy-4, best-buy-5, and so on actually exist!
        Yelp acknowledges this fact, but still returns a BusinessUnavaialable error when queries.
        This method sends a web request and checks the response and terminates on a 404, which has proven to be a reliable
        way of circumnavigating this issue.
    5.  In case the above doesn't work...
        The manual_override parameter forces the fetcher to keep moving past this error.
        For debugging purposes, this method prints a URL for the purposes of manually checking breakpoints.
        That way you can incrementally run fetch() and then comb over trouble spots you find by moving up manual_override.
        If you hit that URL and you get either a valid ID or an invalid but existing ID, you need to bump up manual_override
        to correct it and rerun the fetch.
        If you hit that URL and you get a 404 page then you're done!
        e.x. In the Best Buy case both best-buy-3 and best-buy-10 are phantoms.
        But once we set manual_override=10 we're good, and get all of the actual storefronts.
    """
    i = 2
    # Run the first one through by hand.
    valid_ids = ["{0}-{1}".format(name, area)]
    responses = [client.get_business(valid_ids[0])]
    # The rest are handled by a loop.
    while True:
        bus_id = "{0}-{1}-{2}".format(name, area, i)
        # print(bus_id)
        try:
            response = client.get_business(bus_id)
        except BusinessUnavailable:
            # We manually check trouble spots.
            # But see the TODO.
            if requests.get('http://www.yelp.com/biz/' + bus_id).status_code != requests.codes.ok:
                break
            else:
                # Increment the counter but don't include the troubled ID.
                i += 1
                continue
        # valid_ids += [bus_id]
        responses += [response]
        i += 1
    print("Ended `fetch_businesses()` on:", "http://www.yelp.com/biz/" + bus_id)
    return responses        


def frame(responses):
    """
    Given a list of yelp.obj.business_response.BusinessResponse objects like the one returns by fetch_businesses(),
    builds a coordinate-logging DataFrame out of them.
    """
    latitudes = [response.business.location.coordinate.latitude for response in responses]
    longitudes = [response.business.location.coordinate.longitude for response in responses]
    df = DataFrame({'latitude': latitudes, 'longitude': longitudes})
    df.index.name=responses[0].business.name
    return df


def map_coordinates(df):
    """
    Returns a folium map of all of the coordinates stored in a coordinate DataFrame, like the one returned by frame().
    """
    ret = folium.Map(location=[40.753889, -73.983611], zoom_start=11)
    for row in df.iterrows():
        ret.simple_marker([row[1]['latitude'], row[1]['longitude']])
    return ret


def load_geojson(filename="manhattan.geojson"):
    """
    Returns a geojson object for the given file.
    """
    with open(filename) as f:
        dat = f.read()
        obj = geojson.loads(dat)
    return obj


def load_coordinates(name, filename="manhattan.geojson"):
    """
    Loads Manhattan.
    What else?
    Are you surprised?
    
    Later on it ought to be able to load some named city region.
    For now the paramter is quietly ignored.
    """
    obj = load_geojson(filename)
    return list(geojson.utils.coords(obj))


# Borrowed from: http://www.ariel.com.au/a/python-point-int-poly.html
def point_inside_polygon(x,y,poly):
    """
    Checks if a point is inside a polygon.
    Used to validate points as being inside of Manahttan.
    Borrowed from: http://www.ariel.com.au/a/python-point-int-poly.html
    
    The shapely library provides features for this and other things besides, but is too much to deal with at the moment.
    """

    n = len(poly)
    inside = False

    p1x,p1y = poly[0]
    for i in range(n+1):
        p2x,p2y = poly[i % n]
        if y > min(p1y,p2y):
            if y <= max(p1y,p2y):
                if x <= max(p1x,p2x):
                    if p1y != p2y:
                        xints = (y-p1y)*(p2x-p1x)/(p2y-p1y)+p1x
                    if p1x == p2x or x <= xints:
                        inside = not inside
        p1x,p1y = p2x,p2y

    return inside


def generate_sample_points(coordinate_list, n=1000):
    """
    Generates n uniformly distributed sample points within the given coordinate list.
    
    When the geometry is sufficiently complex and the list of points large this query can take a while to process.
    """
    lats, longs = list(map(lambda coords: coords[0], coordinate_list)), list(map(lambda coords: coords[1], coordinate_list))
    max_lat = max(lats)
    min_lat = min(lats)
    max_long = max(longs)
    min_long = min(longs)
    ret = []
    while True:
        p_lat = random.uniform(min_lat, max_lat)
        p_long = random.uniform(min_long, max_long)
        if point_inside_polygon(p_lat, p_long, coordinate_list):
            ret.append((p_lat, p_long))
            if len(ret) > n:
                break
        else:
            continue
    return ret


def sample_points(search_location, n=10000):
    """
    Given the name of the location being search, returns n uniformally distributed points within that location.
    
    Wraps the above.
    """
    return generate_sample_points(load_coordinates(search_location), n)


def get_minimum_distance(coordinate, coordinate_list):
    """
    Naively calculates the minimum distance in the point cloud.
    """
    best_coord = (0, 0)
    best_distance = 1000
    for candidate_coord in coordinate_list:
        dist = vincenty(coordinate, candidate_coord).miles
        if dist < best_distance:
            best_coord = candidate_coord
            best_distance = dist
    return best_distance


def average_distance(chain_name, search_location, point_cloud):
    """
    This is the main function of this notebook!
    Takes the name of the chain in question and the point cloud associated with the location
    for which we are computing average distance.
    Returns the average distance to that chain within that location.
    We ask for a point cloud and not the name of the location because it's more efficient to precompute an extremely large,
    essentially totally random point cloud, and then check against that, instead of recomputing it every round.
    """
    # First load the coordinates corresponding to the search location..
    location_coords = load_coordinates(search_location)
    # Now generate a list of the chains' locations.
    chain_df = frame(fetch_businesses(chain_name))
    chain_coords = list(zip(chain_df['longitude'], chain_df['latitude']))
    # Finally, get and average the minimum distances between the points in the point cloud and the chain locations.
    distances = [get_minimum_distance(point, chain_coords) for point in point_cloud]
    avg = sum(distances)/len(distances)
    return avg

In [3]:
credentials = import_credentials()

auth = Oauth1Authenticator(
    consumer_key=credentials['consumer_key'],
    consumer_secret=credentials['consumer_secret'],
    token=credentials['token'],
    token_secret=credentials['token_secret']
)

client = Client(auth)

In [4]:
best_buy = fetch_businesses('Best Buy')
map_coordinates(frame(best_buy))

Ended `fetch_businesses()` on: http://www.yelp.com/biz/best-buy-new-york-11


In [5]:
gregorys = fetch_businesses('Gregorys Coffee')
map_coordinates(frame(gregorys))

Ended `fetch_businesses()` on: http://www.yelp.com/biz/gregorys-coffee-new-york-20


In [6]:
starbucks = fetch_businesses('Starbucks')
map_coordinates(frame(starbucks))

Ended `fetch_businesses()` on: http://www.yelp.com/biz/starbucks-new-york-474


In [12]:
manhattan_point_cloud = sample_points("Manhattan", n=1000)

In [13]:
plt.output_notebook(hide_banner=True)

p = plt.figure(height=500,
                width=960,
                title="Manhattan Point Cloud",
                x_axis_label="Latitude",
                y_axis_label="Longitude"
               )

p.scatter(
    [coord[0] for coord in manhattan_point_cloud],
    [coord[1] for coord in manhattan_point_cloud]
#     [1],
#     [2]
)

plt.show(p)

In [8]:
average_distance("Bibble and Sip", "Manhattan", manhattan_point_cloud)

Ended `fetch_businesses()` on: http://www.yelp.com/biz/bibble-and-sip-new-york-3


2.1297145826710793